In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score

# Load the data
train_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training_set_features.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training_set_labels.csv')
test_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_set_features.csv')

# Merge the features and labels
train_data = pd.merge(train_features, train_labels, on='respondent_id')

# Drop the 'respondent_id' as it's not a feature
X = train_data.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
y = train_data[['xyz_vaccine', 'seasonal_vaccine']]
test_data = test_features.drop(columns=['respondent_id'])

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Define the model
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

# Create and evaluate the pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Predict on the validation set
y_pred_valid = clf.predict_proba(X_valid)
y_pred_valid = np.hstack([y_pred_valid[0][:, 1].reshape(-1, 1), y_pred_valid[1][:, 1].reshape(-1, 1)])

# Evaluate the model
roc_auc = roc_auc_score(y_valid, y_pred_valid, average='macro')
print(f'Validation ROC AUC Score: {roc_auc}')

# Predict on the test set
y_pred_test = clf.predict_proba(test_data)
y_pred_test = np.hstack([y_pred_test[0][:, 1].reshape(-1, 1), y_pred_test[1][:, 1].reshape(-1, 1)])

# Prepare the submission file
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': y_pred_test[:, 0],
    'seasonal_vaccine': y_pred_test[:, 1]
})

# Save the submission file
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv', index=False)
print("Submission file saved successfully.")


Validation ROC AUC Score: 0.8406199199127561
Submission file saved successfully.
